In [ ]:
# 本文件为kaggle环境下的脚本文件副本
!git clone https://github.com/Wkingxc/Bert-Vits2-Colab
%cd /kaggle/working/Bert-Vits2-Colab

In [ ]:
# c_name为角色名称，实验目录统一为EX1
c_name = "希儿"
c_voice = "Xier.zip"
# 创建目录
!mkdir -p /kaggle/working/Bert-Vits2-Colab/Data/EX1/filelists/
!mkdir -p /kaggle/working/Bert-Vits2-Colab/Data/EX1/models/
!mkdir -p /kaggle/working/Bert-Vits2-Colab/Data/EX1/raw/$c_name/
!mkdir -p /kaggle/working/Bert-Vits2-Colab/Data/EX1/wavs/
# 从数据集文件夹中拷贝音频和台词文件
!cp /kaggle/input/xier-voice/* /kaggle/working/Bert-Vits2-Colab/Data/EX1/raw/$c_name/

In [ ]:
!pip install -r ./requirements.txt -q

In [ ]:
!echo "=====下载 bert====="
!wget -P ./bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -P ./bert/deberta-v2-large-japanese-char-wwm/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm/resolve/main/pytorch_model.bin
!wget -P ./bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.bin
!wget -P ./bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.generator.bin
!wget -P ./bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/spm.model
!wget -P ./bert/Erlangshen-MegatronBert-1.3B-Chinese https://huggingface.co/IDEA-CCNL/Erlangshen-MegatronBert-1.3B/resolve/main/pytorch_model.bin
# !cp /content/drive/MyDrive/pytorch_model.bin /content/Bert-Vits2-Colab/bert/Erlangshen-MegatronBert-1.3B-Chinese/

!echo "====下载slm===="
!wget -P ./slm/wavlm-base-plus/ https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin
!wget -P ./emotional/clap-htsat-fused https://huggingface.co/laion/clap-htsat-fused/resolve/main/pytorch_model.bin

!echo "====下载中文特化底膜===="
!wget -P "./Data/EX1/models/" https://huggingface.co/guiyun/Bert-VITS2-chinese/resolve/main/D_0.pth
!wget -P "./Data/EX1/models/" https://huggingface.co/guiyun/Bert-VITS2-chinese/resolve/main/G_0.pth
!wget -P "./Data/EX1/models/" https://huggingface.co/guiyun/Bert-VITS2-chinese/resolve/main/WD_0.pth

# 如果是接着训练，请想办法上传这三个文件到models目录下


In [ ]:
# 由于kaggle平台的设备不支持bfloat16，将train_ms.py中的bfloat16替换为float16
with open("train_ms.py", "r") as file:
    content = file.read()
count = content.count("bfloat16")
new_content = content.replace("bfloat16", "float16")
with open("train_ms.py", "w") as file:
    file.write(new_content)
print(f"替换了 {count} 处") # 6处

In [ ]:
# 下面正式开始训练步骤
!nvidia-smi

In [ ]:
# 配置config.json和config.yaml
# 默认500步保存一次，最多保存3个模型
%cd ./Data/EX1/
!python set_batch.py -b 6 # 指定batch_size
!python set_yml.py -b 2 -e 2 -t 2 # 指定bert_gen、clap_gen、train时训练集加载线程数
%cd ../../

In [ ]:
!python trans_wav_lab.py

In [ ]:
!python preprocess_text.py

In [ ]:
!python bert_gen.py

In [ ]:
!python clap_gen.py

In [ ]:
# !python train_ms.py
# 使用双T4卡双线程训练
!torchrun \
    --nnodes=1\
    --nproc_per_node=2\
    train_ms.py